Start with installing required dependencies.

In [4]:
!pip install numpy opencv-python

Import required dependencies.

In [5]:
import cv2
import numpy as np

Define a method to draw a red rectangle in open cv frame as shown in the screenshot from assessment.

In [6]:
def draw_dashed_rectangle(frame, start_point, end_point, color = (0, 0, 255), thickness = 4, dash_length = 15):
    start_x, start_y = start_point
    end_x, end_y = end_point
    
    for x in range(start_x, end_x, dash_length * 2):
        start_point = (x, start_y)
        end_point = (min(x + dash_length, end_x), start_y)
        cv2.line(frame, start_point, end_point, color, thickness)

        start_point = (x, end_y)
        end_point = (min(x + dash_length, end_x), end_y)
        cv2.line(frame, start_point, end_point, color, thickness)

    for y in range(start_y, end_y, dash_length * 2):
        start_point = (start_x, y)
        end_point = (start_x, min(y + dash_length, end_y))
        cv2.line(frame, start_point, end_point, color, thickness)

        start_point = (end_x, y)
        end_point = (end_x, min(y + dash_length, end_y))
        cv2.line(frame, start_point, end_point, color, thickness)

Define a method returning coordinates of the Main Street borders.

In [7]:
def get_main_street_border_coordinates(frame):
    return (
        2,
        int(frame.shape[0] * 0.43),
        frame.shape[1] - 4,
        frame.shape[0] - 10
    )

Define a method for a frame preprocessing. We crop the main street, apply grayscale and Gaussian Blur effects. Such a preprocessing simplifies car detection algorithm and improves its performance allowing to avoid unnecessary image details.

In [8]:
def preprocess_frame(frame, main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y):
    detection_frame = frame[main_street_start_y:main_street_end_y, main_street_start_x:main_street_end_x]
    gray_frame = cv2.cvtColor(detection_frame, cv2.COLOR_BGR2GRAY)
    return cv2.GaussianBlur(gray_frame, (25,25), 1)

Run the car detection demonstration.

In [9]:
# read video file
video = cv2.VideoCapture('Exercise1_Files/Traffic_Laramie_1.mp4')

# read initial frame without cars for using in background subtraction
initial_frame = video.read()[1]

# define main street borders
main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y = get_main_street_border_coordinates(initial_frame)

# preprocess initial frame
initial_frame = preprocess_frame(initial_frame, main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y)

# prepare previous frame for using in frame differencing 
prev_frame = initial_frame

# read the next frame
check, frame = video.read()
while check:
    
    # preprocess the current frame
    blur_frame = preprocess_frame(frame, main_street_start_x, main_street_start_y, main_street_end_x, main_street_end_y)
    
    # calculate "delta" frame using frame differencing and background subtraction techniques at the same time
    delta_frame = (cv2.absdiff(initial_frame, blur_frame) * 0.5 + cv2.absdiff(prev_frame, blur_frame) * 0.5).astype(np.uint8)
    
    # apply threshold to the "delta" frame for ease of objects detection
    threshold_frame=cv2.threshold(delta_frame, 12, 255, cv2.THRESH_BINARY)[1]

    # find objects contours in the threshold frame
    contours, _ = cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # iterate over found objects
    for c in contours:
        
        # skip small objects (people, bicycles, etc.)
        if cv2.contourArea(c) < 3000:
            continue
        
        # find coordinates of a rectangle around the object
        (x, y, w, h) = cv2.boundingRect(c)
        
        # draw rectangle as shown in the screenshot from assessment
        cv2.rectangle(
            frame, 
            (x + main_street_start_x, y + main_street_start_y), 
            (x+ main_street_start_x + w, y+main_street_start_y+h), 
            (0, 255, 0), 
            1
        )

    # draw the street text label as shown in the screenshot from assessment
    cv2.putText(frame, 'Main street', (main_street_start_x, main_street_start_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)

    # draw Main Street borders as shown in the screenshot from assessment
    draw_dashed_rectangle(
        frame = frame,
        start_point = (main_street_start_x, main_street_start_y),
        end_point = (main_street_end_x, main_street_end_y)
    )
    
    # show detected cars
    cv2.imshow('Traffic Camera', frame)

    # update the previous frame
    prev_frame = blur_frame

    # read the next frame
    check, frame = video.read()

    # handle quit
    playbackAction = cv2.waitKey(1)
    if playbackAction == ord('q'):
        break

# gracefully shutdown the demonstration
video.release()
cv2.destroyAllWindows()